In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from scipy.stats import chi2

class Files:
    train = 'train.csv'

# Algorithmically Identifying Feature Engineering Opportunities

Let's briefly demo a method for gaining quick insight into feature relationships with predictive power in mind. I've included motivating ideas at the bottom of the notebook if you're interested.

In [2]:
data = pd.read_csv(Files.train)
target = 'Listening_Time_minutes'
irrelevant_cols = ['id']

X = data.drop(columns = irrelevant_cols + [target])
y = data[target]

In [3]:
def traverse(node: dict, parents: list, ancestors: list) -> None:
    if 'split_feature' in node: # selects non-leaf nodes
        feature = node['split_feature']
        for parent in parents:
            ancestors.append((parent, feature))
        parents = parents.copy()
        parents.append(feature)

        if 'left_child' in node:
            traverse(node['left_child'], parents, ancestors)
        if 'right_child' in node:
            traverse(node['right_child'], parents, ancestors)
            

def tally_results(ancestors: list) -> pd.DataFrame:
    parent_count = {}
    child_count = {}
    tuple_count = {}
    total = 0
    for p, c in ancestors:
        total += 1
        parent_count[p] = 0 if p not in parent_count else parent_count[p] + 1
        child_count[c] = 0 if c not in child_count else child_count[c] + 1
        tuple_count[(p, c)] = 0 if (p, c) not in tuple_count else tuple_count[(p, c)] + 1

    p_vals = {}
    for (p, c), observed in tuple_count.items():
        expected = parent_count[p] * child_count[c] / total
        chi2_stat = (observed - expected)**2 / expected
        p_vals[(p, c)] = 1 - chi2.cdf(chi2_stat, df=1)

    df_items = [{'Parent': p, 'Child': c, 'p_value': val} for (p, c), val in p_vals.items()]
    return pd.DataFrame(df_items)
    

def ranked_feature_relationships(X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
    X, y = X.copy(), y.copy()
    cat_cols = X.select_dtypes('object').columns.to_list()
    X[cat_cols] = X[cat_cols].astype('category')

    features = X.columns.to_list()

    dtrain = lgb.Dataset(X, label=y, categorical_feature=cat_cols)
    params = {
        "objective": "regression",
        "boosting": "gbdt",
        "verbose": -1,
        "max_depth": 3, # feel free to edit: I find shallow trees are better for pulling out key relationships
    }
    model = lgb.train(params, dtrain, num_boost_round=100)

    trees = model.dump_model()["tree_info"]
    ancestors = []
    for tree in trees:
        parents = []
        traverse(tree['tree_structure'], parents, ancestors)

    p_values = tally_results(ancestors)
    p_values[['Parent', 'Child']] = p_values[['Parent', 'Child']].map(lambda x: features[x])
    p_values = p_values.sort_values('p_value', ascending=True).reset_index(drop=True)
    return p_values

In [4]:
p_vals = ranked_feature_relationships(X, y)
p_vals.query('Parent != Child and p_value < 0.05').head(15)

,Parent,Child,p_value
0,Episode_Length_minutes,Number_of_Ads,2.944100e-08
1,Podcast_Name,Episode_Title,9.686866e-04
2,Episode_Length_minutes,Episode_Title,2.770240e-03
3,Episode_Sentiment,Guest_Popularity_percentage,6.981304e-03
4,Host_Popularity_percentage,Number_of_Ads,1.084662e-02
5,Episode_Length_minutes,Guest_Popularity_percentage,1.394573e-02
6,Podcast_Name,Number_of_Ads,1.448542e-02
7,Number_of_Ads,Episode_Title,1.507850e-02
9,Episode_Sentiment,Host_Popularity_percentage,2.232855e-02
10,Episode_Sentiment,Publication_Time,4.768292e-02


This is empirical evidence that we should spend some time looking into combining these feature pairs.

0. The model likes splitting on `Number_of_Ads` after `Episode_Length_minutes` has been refined. This is intuitive; ad frequency is a more important measure.
1. For those who've done some EDA, (`Podcast_Name`, `Episode_Title`) is not uniquely identifying. In fact, each pair has lots of datapoints. We can experiment with feature concatenation here.
2. This is less intuitive; something to explore further... maybe some categorical or target encoding here?
3. Again, less obvious. Maybe listeners who like the guest are willing to put up with a less enjoyable listening experience for longer? Since `Episode_Sentiment` has only three categories, we could try many encoding schemes.
4. Go forth and explore. Adventure is out there!

In [5]:
p_vals.query('Parent == Child and p_value < 0.05').head(15)

,Parent,Child,p_value
8,Podcast_Name,Podcast_Name,0.019126


We see that the model likes to split on `Podcast_Name` more than once. This suggests there are clustering and/or encoding opportunities here.

I'm hoping to build on this in the near future, so let me know if you have any suggestions for improvement.

# Background and Justification

Ordinary metrics like correlation are pretty limited when it comes to identifying feature relationships. A much better metric is something like mutual information, but it relies on distribution approximations. Plus, it's more reliable for binary relationships, like I(X, Y) or I(X1, X2). The first of these is rarely necessary since we can plot Y vs X, and the second doesn't promise application to the problem of predicting Y.

The reason MI is so powerful, especially for something like tabular data, is that it is invariant to monotonic transformations of the variables. Unlike correlation, it doesn't need its variables to be linearly related. Decision trees exhibit this same phenomenon, invariance under monotonic transformations. This, I believe, is the reason for the empirical superiority of decision trees in tabular prediction.

For categorical data, we don't need complex distribution approximations; we can simply get the frequency counts. The chi-squared test is a well-understood empirical method for addressing the question of two categorical features being related. Indeed, it answers the same question as MI: how far does the empirical, 2D-distribution drift from the assumption of independence? Chi-squared tests allow us to easily convert these results into a probability, or p-value.

## Feature Engineering

Feature engineering is an art. We might start with some visualizations, intuitive experiments, domain knowledge, statistics, etc. to help us navigate the endless combinations. But the curse of dimensionality is always with us. This motivates a strategic approach that:
1. Identifies strong feature relationships,
2. Promises performance improvements.


The method described in this notebook for achieving these ends can be summarized as follows:
1. Train a gradient-boosted decision trees model on the data.
2. Traverse the trees, recording every time a split is a direct or indirect descendant of another.
3. Go through the pairs, recording how often each feature is a parent or child.
4. Estimate the expected pair frequency of (p, c) given empirical parent and child counts for p and c, respectively.
5. Perform a chi-squared test with the observed (p, c) counts.
6. Convert to p-values and start feature engineering.

Note that even with both numerical and categorical columns, we're looking only at the feature relationships via the split structure of the trees.

When a split is a descendant of another, the incoming data is filterd by the earlier split. So, when the probability of a split becomes more likely given another (informed by chi-squared), we know that there is mutual information. More importantly, we know that this mutual information is relevant for predicting Y. In terms of mutual information, we're finding those parent-child pairs such that
$$I(X_{child} \text{ }; Y \text{ }|\text{ } X_{parent}) \ge I(X_{child} \text{ }; Y).$$
These are the pairs that promise us the most performance improvement when combined effectively. To be clear, there's still work to be done, but knowing which pairs to investigate closely yields a massive speed-up. In some cases, we can identify opportunities we might not have tried otherwise.